In [1]:
from qiskit.chemistry import FermionicOperator
from qiskit.chemistry.aqua_extensions.components.initial_states import HartreeFock
from qiskit.chemistry.aqua_extensions.components.variational_forms import UCCSD
from qiskit.chemistry import QMolecule as qm

from qiskit.aqua.components.optimizers import COBYLA
from qiskit.aqua import Operator
from qiskit.aqua.algorithms import VQE, ExactEigensolver
from qiskit import Aer
from qiskit.tools.visualization import circuit_drawer

from pytket.qiskit import *

from qiskit.chemistry.drivers import PySCFDriver, UnitsType
import numpy as np
import matplotlib as plt
import yaml as yml
from yaml import SafeLoader as Loader
import Load_Hamiltonians as lh
import os
import time

In [2]:
root_dir = '/Users/mmetcalf/Dropbox/Quantum Embedding/Codes/Lithium_Downfolding/Qiskit Chem/Hamiltonian_Downfolding_IBM/IntegralData/H2_MEKENA/'
NW_data_file = str(root_dir+'h2_ccpvtz_ccsd_0_80au_ducc_1_3.yaml')
# NW_data_file = str('H2.yaml')
OE_data_file = str(root_dir+ 'h2_ccpvtz_ccsd_0_80au.FOCK')
# NW_data_file = 'H2.yaml'
doc_nw = open(NW_data_file,'r')
doc_oe = open(OE_data_file,'r')
data = yml.load(doc_nw,Loader)
content_oe = doc_oe.readlines()
#################
backend1 = Aer.get_backend('statevector_simulator')
backend2 = Aer.get_backend('qasm_simulator')
#################

###########################################################
#Initialize Variables
map_type = str('jordan_wigner')
truncation_threshold = 0.1
n_spatial_orbitals = data['integral_sets'][0]['n_orbitals']
nuclear_repulsion_energy = data['integral_sets'][0]['coulomb_repulsion']['value']
print('repulsion: ',nuclear_repulsion_energy)
n_orbitals = 2*n_spatial_orbitals
n_particles = data['integral_sets'][0]['n_electrons']

#Populating the QMolecule class with the data to make calculations easier
qm.num_orbitals = n_spatial_orbitals
qm.num_alpha = n_particles//2
qm.num_beta = n_particles//2
#This is just a place holder for freezing the core that I need for the MP2 calculations
qm.core_orbitals = 0
qm.nuclear_repulsion_energy = nuclear_repulsion_energy
qm.hf_energy = data['integral_sets'][0]['scf_energy']['value']

###################Get Orbital Energies from FOCK file########################
orbital_energies = []
found = False
count = 0
for line in content_oe:
    if 'Eigenvalues:' in line.split()[0]:
        found =True
    if found and len(line.split()) > 1:
        orbital_energies.append(float(line.split()[1]))
        count += 1
        if count >= n_spatial_orbitals:
            break
qm.orbital_energies = orbital_energies
print(orbital_energies)
#7 orbitals
#qm.orbital_energies = [-2.4533,-2.4530,-0.1817,0.0065, 0.0273, 0.0273, 0.0385]
##############################################################################

dist = 2*data['integral_sets'][0]['geometry']['atoms'][1]['coords'][2]
if map_type == 'parity':
    #For two-qubit reduction
    n_qubits = n_orbitals-2
else:
    n_qubits = n_orbitals

active_occ = n_particles//2
active_virt = (n_orbitals-n_particles)//2
active_occ_list = np.arange(active_occ)
active_occ_list = active_occ_list.tolist()
#print('Orbitals {} are occupied'.format(active_occ_list))
active_virt_list = np.arange(active_virt)
active_virt_list = active_virt_list.tolist()

repulsion:  1.2499999999999998
[-0.7409033, 0.19499881, 0.25575591, 0.69608275]


In [3]:
one_electron_import = data['integral_sets'][0]['hamiltonian']['one_electron_integrals']['values']
two_electron_import = data['integral_sets'][0]['hamiltonian']['two_electron_integrals']['values']

one_electron_spatial_integrals, two_electron_spatial_integrals = lh.get_spatial_integrals(one_electron_import,two_electron_import,n_spatial_orbitals)
one_electron_spatial_integrals, two_electron_spatial_integrals = lh.trunctate_spatial_integrals(one_electron_spatial_integrals,two_electron_spatial_integrals,.001)

qm.mo_eri_ints = two_electron_spatial_integrals
h1, h2 = lh.convert_to_spin_index(one_electron_spatial_integrals,two_electron_spatial_integrals,n_spatial_orbitals, .001)

fop = FermionicOperator(h1, h2)
qop_paulis = fop.mapping(map_type)
qop = Operator(paulis=qop_paulis.paulis)

In [6]:
init_state = HartreeFock(n_qubits, n_orbitals, n_particles, map_type,two_qubit_reduction=False)

var_op = UCCSD(num_qubits=n_qubits, depth=1, num_orbitals=n_orbitals, num_particles=n_particles, 
               active_occupied=active_occ_list, active_unoccupied=active_virt_list,initial_state=init_state, 
               qubit_mapping=map_type, mp2_reduction=True, singles_deletion=True)

print('There are {} params in this anzats'.format(var_op.num_parameters))
dumpy_params = np.random.rand(var_op.num_parameters)
var_cirq = var_op.construct_circuit(dumpy_params)
# print(var_cirq)

Here are the singles:  []
There are 9 are doubles
There are 5 are the new doubles
[<qiskit.aqua.operator.Operator object at 0x122a57f60>, <qiskit.aqua.operator.Operator object at 0x122a57908>, <qiskit.aqua.operator.Operator object at 0x122a57fd0>, <qiskit.aqua.operator.Operator object at 0x122a57668>, <qiskit.aqua.operator.Operator object at 0x122a7f630>]
There are 5 params in this anzats


In [8]:
from qiskit.visualization import circuit_drawer

circuit_drawer(var_cirq, filename='uccsd_circuit_h2.png',output='mpl')

<Figure size 1923.06x29289 with 1 Axes>